In [ ]:
import numpy as np
import pandas as pd
from numba import njit, jit

from math import log, sqrt, pi, exp, erf, ceil, floor
from scipy.stats import norm, pearsonr
from scipy.stats.mstats import gmean

from matplotlib import pyplot as plt
from tqdm import tqdm, trange
from time import perf_counter
import gc

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

""" Plt font size """
plt.rcParams['font.size'] = '13'

# Code from Example

In [ ]:
def initialize_grid(max_iter_time, plate_length, boundary_value):

    # Initialize solution: the grid of u(k, i, j)
    u = np.empty((max_iter_time, plate_length, plate_length))

    # Initial condition everywhere inside the grid
    u_initial = 0.0

    # Boundary conditions (fixed temperature)
    u_top = boundary_value
    u_left = 0.0
    u_bottom = 0.0
    u_right = 0.0

    # Set the initial condition
    u.fill(u_initial)

    # Set the boundary conditions
    u[:, (plate_length-1):, :] = u_top
    u[:, :, :1] = u_left
    u[:, :1, 1:] = u_bottom
    u[:, :, (plate_length-1):] = u_right


    print("\nInitial 2-D grid in spatial dimension for time snapshot t=0 is listed below\n")
    print(u[0,:,:])
    return u

In [ ]:
#Initialize plate length and max time iterations

plate_length = 50
max_iter_time = 500
boundary_value = 100

initial_grid = initialize_grid(max_iter_time, plate_length, boundary_value)

alpha = 2.0
delta_x = 1

# Calculated params (\Delta t should obey the FTCS condition for stability)
delta_t = (delta_x ** 2)/(4 * alpha)
print("\nUsing a timestep size of \Delta t = ", delta_t)
gamma = (alpha * delta_t) / (delta_x ** 2)

In [ ]:
# Calculate u iteratively on the grid based on the equation derived above

def calculate(u):
    for k in range(0, max_iter_time-1, 1):
        for i in range(1, plate_length-1, delta_x):
            for j in range(1, plate_length-1, delta_x):
                u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*u[k][i][j]) + u[k][i][j]

    return u

In [ ]:
def plotheatmap(u_k, k):
  # Clear the current plot figure
    plt.clf()
    plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")
    plt.xlabel("x")
    plt.ylabel("y")

    # This is to plot u_k (u at time-step k)
    plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=0, vmax=100)
    plt.colorbar()
    return plt

In [ ]:
# Calculate final grid
final_grid = calculate(initial_grid)

# Plot the animation for the solution in time steps

def animate(k):
    plotheatmap(final_grid[k], k)


anim = animation.FuncAnimation(plt.figure(), animate, interval=1,frames=max_iter_time, repeat=False)
# anim.save("heat_equation_solution.gif")
anim

# Code 0:

In [ ]:
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def bottom_boundary_condition(K,T,S_min, r, t):
    return np.zeros(t.shape)

def top_boundary_condition(K,T,S_max, r, t):
    return S_max-np.exp(-r*(T-t))*K

def final_boundary_condition(K,T,S):
    return np.maximum(S-K,0)

In [ ]:
def compute_abc( K, T, sigma, r, S, dt, dS ):
    # a = .5 *
    a = -sigma**2 * S**2/(2* dS**2 ) + r*S/(2*dS)
    b = r + sigma**2 * S**2/(dS**2)
    c = -sigma**2 * S**2/(2* dS**2 ) - r*S/(2*dS)
    return a,b,c

def compute_lambda( a,b,c ):
    return scipy.sparse.diags( [a[1:],b,c[:-1]],offsets=[-1,0,1])

def compute_W(a,b,c, V0, VM):
    M = len(b)+1
    W = np.zeros(M-1)
    W[0] = a[0]*V0
    W[-1] = c[-1]*VM
    return W

In [ ]:
from numpy import log as ln
def price_call_explicit( K, T, r, sigma, N, M):
    """"
    N: Time Steps
    M: Price Steps
    """
    # Choose the shape of the grid
    dt = T/N
    S_min = 0
    S_max = K*np.exp(8*sigma*np.sqrt(T))
    dS = (S_max-S_min)/M
    S = np.linspace(S_min,S_max,M+1)  # convert to log space
    # print(S)

    t = np.linspace(0,T,N+1)
    V = np.zeros((N+1,M+1)) #...

    # Set the boundary conditions
    V[:,-1] = top_boundary_condition(K,T,S_max,r,t)
    V[:,0]  = bottom_boundary_condition(K,T,S_max,r,t)
    V[-1,:] = final_boundary_condition(K,T,S)

    # Apply the recurrence relation
    a,b,c  = compute_abc(K,T,sigma,r,S[1:-1],dt,dS)
    # print(a, b, c)
    Lambda = compute_lambda( a,b,c)
    print(Lambda.toarray())
    identity = scipy.sparse.identity(M-1)

    for i in range(N,0,-1):
        W = compute_W(a, b, c, V[i,0], V[i,M])
        # print(W, "\n")  #  ;   break

        # Use `dot` to multiply a vector by a sparse matrix
        V[i-1,1:M] = (identity-Lambda*dt).dot( V[i,1:M] ) - W*dt

    return V, t, S

In [ ]:
V, t, S = price_call_explicit( K=99, T=1., r=.05, sigma=.2, N=10, M=10)
V

In [ ]:
S_, T_, V_ = [], [], []
for _row in range(V.shape[0]):
    for _column in range(V.shape[1]):
        S_.append(S[_column])
        T_.append(t[_row])
        V_.append(V[_row, _column])

S_, T_, V_ = np.array(S_), np.array(T_), np.array(V_)
S_.shape, T_.shape, V_.shape


In [ ]:
""" Getting Option Values from V """
# 1st Dimension = t
# 2nd Dimension = S value
# So to get the Value of an Option at t=0 for about ~100 : Based on the S vector

""" Get Value """
# V[0, 10]
S_, T_, V_

In [ ]:
t

In [ ]:
S

In [ ]:
# gc.collect()
""" 3D Graph """
plt.rcParams['font.size'] = '15'

fig = plt.figure(figsize=(15,12))
ax = plt.axes(projection='3d')
ax.plot_trisurf(T_, S_, V_, cmap='viridis', edgecolor='none')  # 'viridis' , cm.jet

# plt.xticks(plt.xticks()[0][1:-1], [f'{10**x*100:.2f}' for x in plt.xticks()[0][1:-1]])

ax.set_ylabel('\n\nStock Value (USD)')   ;   ax.set_xlabel('\n\n Time (t) Years')   ;   ax.set_zlabel('\n\n\nOption Value (USD)')
# ax.dist = 10
ax.set_title("Option Value for Different Parameters")


# Code 0.1

In [ ]:
%matplotlib inline
# import numpy
import matplotlib.pyplot as pyplot
from matplotlib import rcParams
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'

T = 1.       # expiry time
r = 0.06        # no-risk interest rate
sigma = 0.2    # volatility of underlying asset
E = 99.        # exercise price
S_max = 4*E    # upper bound of price of the stock (4*E)

In [ ]:
def FTCS(C, N, M, dt, r, sigma):
    """using forward-time central-space scheme to solve the Black-Scholes equation for the call option price

    Arguments:
        C:       array of the price of call option
        N:       total number of time steps
        M:       total number of spatials grids
        dt:      time step
        r:       no-risk interest rate
        sigma:   volatility of the stock

    Returns:
        C:       array of the price of call option
    """
    index = np.arange(1,M)
    for n in range(N):

        C[1:-1] = 0.5 * (sigma**2 * index**2 * dt - r*index*dt) * C[0:-2] \
             +       (1 - sigma**2* index**2 *dt - r*dt) * C[1:-1]   \
             + 0.5 * (sigma**2 * index**2 * dt + r*index*dt) * C[2:]
    return C

# def FTCS_2(C, N, M, dt, r, sigma):
M, dt, r, sigma = 10, .1, .05, .2
index = np.arange(1,M)

B = np.zeros((M,M), dtype=float)
C = np.zeros((M,M), dtype=float)
for m in range(M):
    """ B - forward """
    B[m, m] = 1 - (1 - sigma**2* m**2 *dt - r*dt)
    if m > 0 :
        B[m-1, m] = - 0.5 * (sigma**2 * m**2 * dt + r*m*dt)
    if m != M - 1:
        B[m+1, m] = - 0.5 * (sigma**2 * m**2 * dt - r*m*dt)
    
    """ C - current """
    C[m, m] = 1 + (1 - sigma**2* m**2 *dt - r*dt)
    if m > 0 :
        C[m-1, m] = + 0.5 * (sigma**2 * m**2 * dt + r*m*dt)
    if m != M - 1:
        C[m+1, m] = + 0.5 * (sigma**2 * m**2 * dt - r*m*dt)
    
    
    
# B = np.arange(1,17).reshape((4,4))
# B[1,0]

B
C

# """
#     a = 0.5 * (sigma**2 * index**2 * dt - r*index*dt)
#     B = (1 - sigma**2* index**2 *dt - r*dt)
#     C = 0.5 * (sigma**2 * index**2 * dt + r*index*dt)
# """

# for n in range(N):


In [ ]:
N = 2000       # number of time steps
M = 200        # number of space grids
dt = T/N       # time step
s = np.linspace(0, S_max, M+1)   # spatial grid (stock's price)

# initial condition & boundary condition
C = s - E
C = np.clip(C, 0, S_max-E)

In [ ]:
C_exp = FTCS(C, N, M, dt, r, sigma)
print('the price of the call option should be around {}, \
if the current price of stock is 20 dollar.'.format(C_exp[int(M/2)]))

In [ ]:
C_exp